In [1]:
!pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ifeanyinneji","key":"68b7b1b6c0d102f7d9941d0b3eff800a"}'}

In [2]:
# This cell creates a .kaggle hidden folder in our root directory and copies the kaggle.json file to the folder.
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [3]:
# This cell will download the dataset used for this project to the /content directory and also unzip the csv file.
!kaggle datasets download -d arashnic/book-recommendation-dataset
!unzip /content/book-recommendation-dataset.zip

 21% 5.00M/23.8M [00:00<00:00, 31.8MB/s]
100% 23.8M/23.8M [00:00<00:00, 105MB/s] 
Archive:  /content/book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: Ratings.csv             
  inflating: Users.csv               


In [6]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


dfbooks = pd.read_csv("Books.csv")
dfratings = pd.read_csv("Ratings.csv")
dfusers = pd.read_csv("Users.csv")

# Verify the Dataset Number of Rows and Columns
print("Books Shape: ", dfbooks.shape)
print("Ratings Shape: ", dfratings.shape)
print("Users Shape: ", dfusers.shape)

Books Shape:  (271360, 8)
Ratings Shape:  (1149780, 3)
Users Shape:  (278858, 3)


In [7]:
print('='*20, ' COLUMNS ', '='*20)
print('\n', '-'*20, '1. Books ', '-'*20)
print(dfbooks.columns)

print('\n', '-'*20, '2. Ratings ', '-'*20)
print(dfratings.columns)

print('\n', '-'*20, '3. Users ', '-'*20)
print(dfusers.columns)


print('\n\n')
print('='*20, ' MISSING VALUES ', '='*20)
print('\n', '-'*20, '1. Books ', '-'*20)
print(dfbooks.isna().sum() )

print('\n', '-'*20, '2. Ratings ', '-'*20)
print(dfratings.isna().sum() )

print('\n', '-'*20, '3. Users ', '-'*20)
print(dfusers.isna().sum() )


====================  COLUMNS  ====================

 -------------------- 1. Books  --------------------
Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

 -------------------- 2. Ratings  --------------------
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

 -------------------- 3. Users  --------------------
Index(['User-ID', 'Location', 'Age'], dtype='object')



====================  MISSING VALUES  ====================

 -------------------- 1. Books  --------------------
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

 -------------------- 2. Ratings  --------------------
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

 -------------------- 3. Users  --------------------
User-ID       

In [9]:
# Merging Books and Rating Dataset for UserID and Rating Columns
dfbooks_rating = dfbooks.merge(dfratings, on = 'ISBN')
print("\nShape: ", dfbooks_rating.shape )

print("\n\n")
print('\n', '-'*15, ' Dataset Info ', '-'*15, '\n')
dfbooks_rating.info()

print("\n\n")
print('\n', '-'*30, ' Dataset ', '-'*30, '\n')
dfbooks_rating.head()


Shape:  (1031136, 10)




 ---------------  Dataset Info  --------------- 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   ISBN                 1031136 non-null  object
 1   Book-Title           1031136 non-null  object
 2   Book-Author          1031135 non-null  object
 3   Year-Of-Publication  1031136 non-null  object
 4   Publisher            1031134 non-null  object
 5   Image-URL-S          1031136 non-null  object
 6   Image-URL-M          1031136 non-null  object
 7   Image-URL-L          1031132 non-null  object
 8   User-ID              1031136 non-null  int64 
 9   Book-Rating          1031136 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 86.5+ MB




 ------------------------------  Dataset  ------------------------------ 



,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [10]:
dfpopular_books = dfbooks_rating.groupby('Book-Title').agg(['count', 'mean'])['Book-Rating'].reset_index()
xdfpopular_books = dfpopular_books[dfpopular_books['count'] >= 250 ].sort_values(by='mean', ascending=False).head(50)

dfbooks_popular_final = xdfpopular_books.merge(dfbooks, on="Book-Title").drop_duplicates("Book-Title")
dfbooks_popular_final = dfbooks_popular_final[['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-M', 'count', 'mean']]
dfbooks_popular_final.rename(columns={'count' : 'num_ratings', 'mean' : 'avg_rating'}, inplace=True)

print("Shape: ", dfbooks_popular_final.shape)
dfbooks_popular_final

Shape:  (50, 6)


,Book-Title,Book-Author,Publisher,Image-URL-M,num_ratings,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,Del Rey,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,Del Rey,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,Del Rey,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,To Kill a Mockingbird,Harper Lee,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


## Collaborating Filtering
Technique in which filter items that a user might like on the basis of by similar users

In [11]:
dfbooks_rating_count = dfbooks_rating.groupby('User-ID').agg(['count'])['Book-Rating'].reset_index()
xdfbooks_rating_userID = dfbooks_rating_count[dfbooks_rating_count['count'] > 200] # Count value more than 200

# Filteing the required User of Dataset (Count more than 200 )
xdfbooks_rating = dfbooks_rating[dfbooks_rating['User-ID'].isin(xdfbooks_rating_userID['User-ID'].tolist() )]

print("Shape: ", xdfbooks_rating.shape)
xdfbooks_rating.head()

Shape:  (474007, 10)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
6,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,0
7,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,0
10,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,177458,0
21,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,110912,10


In [13]:
xdfbooks_count = xdfbooks_rating.groupby('Book-Title').agg(['count'])['Book-Rating'].reset_index()
xdfbooks_popular = xdfbooks_count[xdfbooks_count['count'] >= 50]
xdfbooks_famous = xdfbooks_rating[xdfbooks_rating['Book-Title'].isin(xdfbooks_popular['Book-Title'].tolist() )].drop_duplicates()
print("Shape: ", xdfbooks_famous.shape)
xdfbooks_famous.head()

Shape:  (58586, 10)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113519,0


In [14]:
xdf_pivot = xdfbooks_famous.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
xdf_pivot.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
xdf_pivot.fillna(0, inplace=True)
xdf_pivot.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(xdf_pivot)

In [17]:
print("Shape: ", similarity_score.shape)
similarity_score

Shape:  (706, 706)


array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [19]:
import numpy as np
np.where(xdf_pivot.index == 'Zoya')[0][0]

704

In [20]:
# Verify the First Record of Similarity Score
sorted(list(enumerate(similarity_score[0])), key=lambda x:x[1], reverse=True)[1:6]

[(47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397),
 (551, 0.2262639743141286)]

In [21]:
# Input the Name of Book While it returns the Similar Book Name
# with its respective information like Author Name and Image URL

def book_recommender(bk_name):
    # fetch Book Index
    indx = np.where(xdf_pivot.index == bk_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[indx])), key=lambda x:x[1], reverse=True)[1:11]
    data = []
    for i in similar_books:
        item = []
        temp_df = dfbooks[dfbooks['Book-Title'] == xdf_pivot.index[i[0]] ]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values ))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values ))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values ))
        
        data.append(item)
    
    return data

In [22]:
# Verfiy the Function with Parameter as Book Name
# 1984 is the Book Name
book_recommender('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'http://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg'],
 ['Fahrenheit 451',
  'Ray Bradbury',
  'http://images.amazon.com/images/P/3257208626.01.MZZZZZZZ.jpg'],
 ['The Catcher in the Rye',
  'J.D. Salinger',
  'http://images.amazon.com/images/P/0316769487.01.MZZZZZZZ.jpg'],
 ['Naked',
  'David Sedaris',
  'http://images.amazon.com/images/P/0316777730.01.MZZZZZZZ.jpg'],
 ['The Hundred Secret Senses',
  'Amy Tan',
  'http://images.amazon.com/images/P/0399141146.01.MZZZZZZZ.jpg']

Successfully get the Recommended Book Name with repective informations by calling the book_recommender Function against the Book Name (1984) as parameter provided

In [32]:
# Verify Image URL of the Book of 0 index
dfbooks_popular_final['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

In [26]:
# Import Pickle Library for Exporting Dataset 
import pickle
# For Popularity Base [Set of 50-Books]
# Export and Save with Name 'popular.pkl' and write in binary
pickle.dump(dfbooks_popular_final, open('popular.pkl', 'wb'))


# For Colaborative Filter Base
# Export and Save 3-File that uses in Colaborative Filtering
pickle.dump(xdf_pivot, open('xdf_pivot.pkl', 'wb'))
pickle.dump(dfbooks, open('dfbooks.pkl', 'wb'))
pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))